In [ ]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import firedrake
import icepack, icepack.plot, icepack.models

# Larsen Ice Shelf

This demo will involve using real data for the Larsen Ice Shelf in the Antarctic Peninsula.
The use of real data will mostly change how we set up the simulation.
The simulation itself -- involving successive prognostic and diagnostic solves of the physics model -- is virtually identical to what we saw in the last demo.

To access the data we'll use, you'll need to have a login for [EarthData](https://urs.earthdata.nasa.gov/), the web portal through which NASA makes remote sensing data available to the public.
Most of the ice sheet remote sensing data produced by American research institutions is hosted at the [National Snow and Ice Data Center (NSIDC)](https://www.nsidc.org) and an EarthData login is necessary to access data from NSIDC.
In particular, we'll use a velocity map of Antarctica produced as part of the MEaSUREs program, which you can read more about [here](https://nsidc.org/data/nsidc-0484).
For the ice thickness we'll use the [bedmap2](https://www.bas.ac.uk/project/bedmap-2/) data set, which is available from the British Antarctic Survey.
Finally, we'll need a description of the computational domain.
In this case, I created a vector data file of the outline of the Larsen Ice Shelf by tracing over satellite imagery in a [geographic information system](https://en.wikipedia.org/wiki/Geographic_information_system).

Rather than manually download these data sets from the websites they're hosted on, we'll call a few functions in the module `data.py` in this directory to fetch them for us.
(Internally, these functions use a library called [pooch](https://github.com/fatiando/pooch) which handles things like caching the data so it doesn't get downloaded twice, unzipping archived files, and so forth.)
One we have the gridded data sets we'll use the library [rasterio](https://rasterio.readthedocs.io/en/stable/) to read them.
Both pooch and rasterio will have been installed along with icepack, so you don't need to do this yourself.
You will need to have a working installation of [gmsh](https://gmsh.info) in order to generate the mesh.

First, we'll fetch the mesh and print out the path to the file where it's been stored.

In [ ]:
import data
mesh_filename = data.fetch_larsen_mesh()
print(mesh_filename)

This function will put the mesh in a predictable location on your system so it can be found easily later, and it will only download the file the first time you ask for it.
The caching aspect will come in handy because we'll be using much of the same data in the fourth demo.
The data module contains functions to fetch the remaining data, which we'll see below.

### Geometry

Now that we've downloaded the mesh we need to read it.
Firedrake has built-in functions for reading meshes in a variety of formats.
The following code reads in the Larsen mesh and makes a plot of it so that we can see all the boundary IDs.
Boundary segments 1 and 3 correspond to the calving terminus; segment 2 borders the Gipps Ice Rise.

In [ ]:
mesh = firedrake.Mesh(mesh_filename)

fig, axes = icepack.plot.subplots()
axes.set_xlabel('meters')
axes.grid()
icepack.plot.triplot(mesh, axes=axes, linewidth=2)
plt.show(fig)

### Input data

Next, we have to load the input data, starting with the ice thickness.
The bedmap2 dataset that we use actually includes several fields -- thickness, surface elevation, bed elevation, an ice shelf and rock mask, etc.
The function `fetch_bedmap2` will download the dataset from the internet, unzip it, and return a list of the paths of all the files it retrieved rather than just a single file like the `fetch_larsen_mesh` function did.

In [ ]:
bedmap2_files = data.fetch_bedmap2()
for filename in bedmap2_files:
    print(filename)

We're only interested in the thickness data itself, so the following command will pull it out from the list of all the other files in the bedmap2 dataset.

In [ ]:
thickness_filename = [f for f in bedmap2_files if
                      os.path.basename(f) == 'bedmap2_thickness.tif'][0]

The thickness data are stored in a [GeoTIFF](https://en.wikipedia.org/wiki/GeoTIFF) file.
GeoTIFF is a common storage format for geospatial data; it adds georeferencing information on top of the TIFF file format, which is often used for lossless compression of images.
The function `rasterio.open` will give us an object representing the raster data set that we can then read from.

In [ ]:
thickness = rasterio.open(thickness_filename, 'r')

Opening the file doesn't immediately read all the data at once.
Instead, that occurs through calls to the `read` method of the data set, in our case the variable `thickness`.
The actual reading will occur entirely within the functions that interpolate the raster data to our computational mesh.
This two-step procedure involves a little extra code, but the nice part is that you can read only as small a chunk of the data as need be.
It would be awfully wasteful if you had to load up the thickness of all of Antarctica only to focus on a little piece like the Larsen Ice Shelf.

Before going on, let's make a plot of the thickness data.
Plotting all your data first is a good sanity check to make sure that it isn't, say, stored upside-down, or in a different coordinate system than you expected.
To visualize our data, we'll use the function `contourf` in the module `icepack.plot`, which works just like the equivalent matplotlib function.

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.contourf(thickness, 40, axes=axes)
fig.colorbar(contours, label='meters')
plt.show(fig)

Next, we have to fetch the velocity data, which are hosted on NSIDC.
The following will prompt for your EarthData login if need be.
The file is ~6GiB, so if you run this demo yourself, this step could take a while.

In [ ]:
velocity_filename = data.fetch_measures_antarctica()

The velocity data are stored in a [NetCDF](https://en.wikipedia.org/wiki/NetCDF) file.
NetCDF is another common storage format for geophysical data, especially in atmospheric science.
NetCDF offers much more freedom than GeoTIFF in terms of what kind of data can be stored, so you have to know something about the schema or layout before you use it.
For example, many fields can be stored by name in a NetCDF file, and you have to know what all the names are.
The script `ncinfo` will print out information about all the fields stored in a NetCDF file.

In [ ]:
!ncinfo "$velocity_filename"

The fields we want are `VX` and `VY`.
We can use rasterio can read NetCDF files too, but we have to add in a bit of extra magic so it'll know that we want to get the `VX` and `VY`.
To specify which field we're reading, we can prepend `netcdf:` to the beginning of the filename and append `:FIELD_NAME` to the string we pass to `rasterio.open`.

In [ ]:
vx = rasterio.open('netcdf:' + velocity_filename + ':VX', 'r')
vy = rasterio.open('netcdf:' + velocity_filename + ':VY', 'r')

### Modeling

Having done all the leg work to make a mesh and get a good set of input data, the modeling itself should be fairly familiar from the last step.
We'll assume that the ice temperature is a uniform $-13^\circ$C.

One thing is substantially different from previous examples.
Before, we called the function `firedrake.SpatialCoordinate` to get some symbolic handles `x, y` for the mesh coordinates, and we created symbolic expressions to define the input data to our problem analytically.
When we work with real data, we instead use icepack's `GridData` object, which firedrake doesn't know how to interpolate.
The function `icepack.interpolate` works as a layer on top of the firedrake interpolate function and knows what to do with gridded data sets.

In [ ]:
degree = 2
Q = firedrake.FunctionSpace(mesh, family='CG', degree=degree)
V = firedrake.VectorFunctionSpace(mesh, family='CG', degree=degree)

h0 = icepack.interpolate(thickness, Q)
u0 = icepack.interpolate((vx, vy), V)

In [ ]:
fig, axes = icepack.plot.subplots()
streamlines = icepack.plot.streamplot(u0, precision=1000, density=2500, axes=axes)
fig.colorbar(streamlines, label='meters/year')
plt.show(fig)

In [ ]:
T = 260
A = firedrake.interpolate(firedrake.Constant(icepack.rate_factor(T)), Q)

ice_shelf = icepack.models.IceShelf()
opts = {'dirichlet_ids': [2, 4, 5, 6, 7, 8], 'tol': 1e-6}
u = ice_shelf.diagnostic_solve(u0=u0, h=h0, A=A, **opts)

In [ ]:
fig, axes = icepack.plot.subplots()
streamlines = icepack.plot.streamplot(u, precision=1000, density=2500, axes=axes)
fig.colorbar(streamlines, label='meters/year')
plt.show(fig)

We get a fairly reasonable approximation for the velocity even with a spatially homogeneous guess for the ice temperature.

In [ ]:
print(icepack.norm(u - u0) / icepack.norm(u0))

Ballpark estimate, the surface and basal mass balance of Larsen C are +30 and -30 cm/yr respectively, so we can take the total to be 0.
Let's simulate the evolution of the ice shelf for the next 10 years.
The code for this loop should be familiar from the previous example.

In [ ]:
a = firedrake.Function(Q)
h = h0.copy(deepcopy=True)

dt = 0.5
for n in range(int(10 / dt) + 1):
    h = ice_shelf.prognostic_solve(dt, h0=h, a=a, u=u, h_inflow=h0)
    u = ice_shelf.diagnostic_solve(u0=u, h=h, A=A, **opts)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(h, 96, axes=axes)
fig.colorbar(contours, label='meters')
plt.show(fig)

By plotting the difference between the modeled thickness after 10 years and the initial thickness, we can see the propagation of the rifts downstream.
This effect is best visualized with a diverging colormap that makes the 0-contour really obvious.

In [ ]:
δh = firedrake.Function(Q)
δh.assign(h - h0)

fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(δh, axes=axes, cmap='RdBu',
                                    levels=np.linspace(-10, +10, 101), extend='both')
fig.colorbar(contours, label='meters')
plt.show(fig)

The oscillatory pattern makes it less than obvious whether the ice shelf gained or lost mass, so let's evaluate the integral of the thickness change to see.

In [ ]:
from firedrake import assemble, dx
print(assemble(δh * dx) / assemble(1 * dx(mesh)))

Seeing as the simulation ran for 10 years, this isn't a wildly unrealistic number.

### Conclusion

In the last demo, we showed how to simulate ice shelf flow using synthetic data.
Here we showed how to load in a generated mesh and observational data, and we used this same functionality to simulate a real ice shelf.

Many real data sets require some amount of preprocessing before they can be used for modeling.
For example, many velocity data sets have missing pixels or patches due to noise in the optical or radar imagery, and these missing points have to be filled in somehow.
The Bedmap2 thickness also contains processing artifacts that are visible as depressions running diagonally across the ice shelf.
These artifacts could be removed by using a low-pass filter on the gridded data, although this might also wash out some real features like the many rifts in the ice.

In order to run the simulation, we had to come up with a guess for the ice rheology.
The simple choice we made is quite far from the real value and in a subsequent demo we'll show how to estimate it from observational data.